In [2]:
import torch.optim as optim
from torch.utils.data import DataLoader
import torch
import pandas as pd
from utils import *
from Models import GRUModel
from Datasets import myDataset
from CustomMSE import CustomMSE

# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
Using device: cuda


## Prepare the planned path

In [2]:
planned_path = get_planned_path()
path_values = list(planned_path.values())

 Retrying in 1 seconds
***Starting server on port 53453 ***
Loading net-file from './config/osm.net.xml.gz' ... done (107ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 10777.00
Reason: TraCI requested termination.
Performance: 
 Duration: 9.03s
 TraCI-Duration: 5.34s
 Real time factor: 1193.6
 UPS: 129721.785358
Vehicles: 
 Inserted: 2199
 Running: 0
 Waiting: 0
Statistics (avg of 2199):
 RouteLength: 4984.74
 Speed: 9.48
 Duration: 532.63
 WaitingTime: 19.92
 TimeLoss: 76.98
 DepartDelay: 0.53



## Prepare the dataset

In [3]:
# Example DataFrame loading
df = pd.read_csv('edinburgh_trajectories.csv')
num_cols = df.shape[1]
position_indices = [i for i in range(num_cols) if i % 4 == 1 or i % 4 == 2]
position_df = df.iloc[:, position_indices]
position_array = position_df.to_numpy()
sequence_length = len(position_indices) // 2
tensor_list = []

for row in position_array:
    reshaped_tensor = torch.tensor(row.reshape(sequence_length, 2))
    tensor_list.append(reshaped_tensor)

all_trajectories_tensor = torch.stack(tensor_list) / 10

# split the data into training and validation sets
# because the data is randomly generated, we don't need to shuffle it
train_ratio = 0.8
train_size = int(train_ratio * all_trajectories_tensor.shape[0])
train_trajectories_tensor = all_trajectories_tensor[:train_size]
val_trajectories_tensor = all_trajectories_tensor[train_size:]

train_path_values = path_values[:train_size]
val_path_values = path_values[train_size:]

train_mask = generate_masks(train_trajectories_tensor)

train_dataset = myDataset(train_trajectories_tensor, train_mask)
val_dataset = myDataset(val_trajectories_tensor, generate_masks(val_trajectories_tensor))

## Training without masks

In [8]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            for t in range(1, seq_len):
                prediction, hidden = model(current_input, hidden)

                previous_input = inputs[:, t-1, :]
                current_input = inputs[:, t, :].unsqueeze(1)
                loss += criterion(prediction - previous_input, inputs[:, t, :]-previous_input) #(current_input-previous_input).squeeze(1))
                
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

model = GRUModel(input_size=2, hidden_size=128, num_layers=2, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 50, optimizer, criterion)

Epoch 1, Loss: 76211896.57142857
Epoch 2, Loss: 72125614.85714285
Epoch 3, Loss: 64444929.428571425
Epoch 4, Loss: 53451053.28571428
Epoch 5, Loss: 40681944.28571428
Epoch 6, Loss: 29359602.5
Epoch 7, Loss: 20838262.0
Epoch 8, Loss: 15602431.464285715
Epoch 9, Loss: 13097123.857142856
Epoch 10, Loss: 11816173.17857143
Epoch 11, Loss: 9673895.5
Epoch 12, Loss: 7466969.017857143
Epoch 13, Loss: 6517010.178571428
Epoch 14, Loss: 5899727.928571428
Epoch 15, Loss: 5209974.919642857
Epoch 16, Loss: 4263599.330357143
Epoch 17, Loss: 3047009.660714286
Epoch 18, Loss: 1924101.736607143
Epoch 19, Loss: 1154431.6205357143
Epoch 20, Loss: 730893.15625
Epoch 21, Loss: 503118.20870535716
Epoch 22, Loss: 374756.07198660716
Epoch 23, Loss: 307291.26116071426
Epoch 24, Loss: 252715.22544642858
Epoch 25, Loss: 229966.73325892858
Epoch 26, Loss: 205786.9525669643
Epoch 27, Loss: 189376.4966517857
Epoch 28, Loss: 182599.70814732142
Epoch 29, Loss: 177521.2416294643
Epoch 30, Loss: 172630.48214285713
Epoch

## This is used to check bugs
We print a predicted path and the true path.

In [9]:
def checkModel(model, inputs, masks):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            prediction, hidden = model(current_input, hidden)
            current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            print(prediction[2], inputs[:, t, :][2])

# get the first batch in dataloader
val_mask = generate_masks(val_trajectories_tensor, missing_ratio=1, complete_traj_ratio=0)
val_dataset = myDataset(val_trajectories_tensor, val_mask)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
inputs, masks = next(iter(val_dataloader))

checkModel(model, inputs, masks)

tensor([112.3821, 276.8152], device='cuda:0') tensor([152.7798, 242.7674], device='cuda:0')
tensor([150.7395, 228.2400], device='cuda:0') tensor([152.4916, 242.7893], device='cuda:0')
tensor([153.3899, 230.6887], device='cuda:0') tensor([151.9495, 242.8304], device='cuda:0')
tensor([152.8062, 233.3929], device='cuda:0') tensor([151.2525, 242.8855], device='cuda:0')
tensor([150.5053, 232.6945], device='cuda:0') tensor([150.3951, 242.9564], device='cuda:0')
tensor([149.1607, 231.9147], device='cuda:0') tensor([149.3271, 243.0596], device='cuda:0')
tensor([148.1330, 231.1327], device='cuda:0') tensor([148.0590, 243.2058], device='cuda:0')
tensor([147.2610, 230.3664], device='cuda:0') tensor([146.6751, 243.3807], device='cuda:0')
tensor([146.4175, 229.5923], device='cuda:0') tensor([145.3320, 243.6063], device='cuda:0')
tensor([145.5487, 228.7947], device='cuda:0') tensor([143.9566, 243.8790], device='cuda:0')
tensor([144.6805, 227.9920], device='cuda:0') tensor([142.6499, 244.1351], devic

## Evaluate the model (trained without masks)
The validation loss is in meters: how far is the predicted position from the actual position.

5 values refers to the loss when 90% of data is missing, 80% of data is missing, 70% of data is missing, 50% of data is missing.

In [10]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            prediction, hidden = model(current_input, hidden)
            current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((prediction - inputs[:, t, :]) * 10, dim=1) * (inputs[:, t, :] != 0)[:, 0]
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

for i in [0.9, 0.8, 0.7, 0.6, 0.5]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = myDataset(val_trajectories_tensor, val_mask)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

Validation Loss: 214.68404884252496
Validation Loss: 108.95511669273205
Validation Loss: 77.0106295220582
Validation Loss: 58.15108454207984
Validation Loss: 47.33588795607266


## Training with masks

In [11]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            for t in range(1, seq_len):
                prediction, hidden = model(current_input, hidden)

                previous_input = inputs[:, t-1, :]
                if epoch > 30:
                    current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
                else:
                    current_input = inputs[:, t, :].unsqueeze(1)
                loss += criterion(prediction - previous_input, inputs[:, t, :]-previous_input) #(current_input-previous_input).squeeze(1))
                
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

model = GRUModel(input_size=2, hidden_size=128, num_layers=2, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 50, optimizer, criterion)

Epoch 1, Loss: 76042721.42857143
Epoch 2, Loss: 72820772.0
Epoch 3, Loss: 67505544.71428572
Epoch 4, Loss: 60039397.28571428
Epoch 5, Loss: 52484033.571428575
Epoch 6, Loss: 45361327.0
Epoch 7, Loss: 40023337.85714286
Epoch 8, Loss: 35654083.71428572
Epoch 9, Loss: 30136567.0
Epoch 10, Loss: 16502354.464285715
Epoch 11, Loss: 8561739.642857144
Epoch 12, Loss: 5946322.482142857
Epoch 13, Loss: 5201181.3125
Epoch 14, Loss: 4734189.955357143
Epoch 15, Loss: 4080952.848214286
Epoch 16, Loss: 3328033.589285714
Epoch 17, Loss: 2523721.160714286
Epoch 18, Loss: 1791670.169642857
Epoch 19, Loss: 1196116.3504464286
Epoch 20, Loss: 799139.7633928572
Epoch 21, Loss: 565083.6071428572
Epoch 22, Loss: 414113.79910714284
Epoch 23, Loss: 322361.1953125
Epoch 24, Loss: 268059.2823660714
Epoch 25, Loss: 235901.0189732143
Epoch 26, Loss: 220940.36830357142
Epoch 27, Loss: 204528.71484375
Epoch 28, Loss: 195513.0385044643
Epoch 29, Loss: 187031.65457589287
Epoch 30, Loss: 181200.3203125
Epoch 31, Loss: 1

## Test and Eval
Since the method is different (masked training), we need new methods to test.

In [12]:
def checkModel(model, inputs, masks):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            prediction, hidden = model(current_input, hidden)
            current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            print(prediction[2], inputs[:, t, :][2])

# get the first batch in dataloader
val_mask = generate_masks(val_trajectories_tensor, missing_ratio=1, complete_traj_ratio=0)
val_dataset = myDataset(val_trajectories_tensor, val_mask)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
inputs, masks = next(iter(val_dataloader))

checkModel(model, inputs, masks)

tensor([ 68.7743, 291.3376], device='cuda:0') tensor([104.3572, 249.7872], device='cuda:0')
tensor([ 76.7479, 252.5351], device='cuda:0') tensor([104.1524, 249.3824], device='cuda:0')
tensor([ 92.8822, 250.3779], device='cuda:0') tensor([103.8580, 248.8004], device='cuda:0')
tensor([ 91.7657, 251.4802], device='cuda:0') tensor([103.4805, 248.0542], device='cuda:0')
tensor([ 87.5764, 253.9184], device='cuda:0') tensor([103.0184, 247.1407], device='cuda:0')
tensor([ 87.6342, 256.5693], device='cuda:0') tensor([102.4401, 245.9976], device='cuda:0')
tensor([ 89.1432, 259.0818], device='cuda:0') tensor([101.7575, 244.6482], device='cuda:0')
tensor([ 90.7012, 261.4856], device='cuda:0') tensor([101.1195, 243.3870], device='cuda:0')
tensor([ 91.9857, 263.7145], device='cuda:0') tensor([100.4322, 242.0285], device='cuda:0')
tensor([ 93.0166, 265.7896], device='cuda:0') tensor([ 99.7591, 240.6979], device='cuda:0')
tensor([ 93.8494, 267.7281], device='cuda:0') tensor([ 99.0748, 239.3451], devic

In [13]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            prediction, hidden = model(current_input, hidden)
            current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((prediction - inputs[:, t, :]) * 10, dim=1) * (inputs[:, t, :] != 0)[:, 0]
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

for i in [0.9, 0.8, 0.7, 0.6, 0.5]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = myDataset(val_trajectories_tensor, val_mask)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

Validation Loss: 233.69360424149724
Validation Loss: 129.92423028339664
Validation Loss: 90.2996298571846
Validation Loss: 69.43734962276797
Validation Loss: 57.116678231391894
